<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/LabSlope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
hemo = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Hemodialysis.csv', encoding='latin1')
hemo.shape

(149987, 5)

In [3]:
import pandas as pd
lab1 = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Lab.csv')
lab1.shape

(21641523, 11)

In [4]:
import pandas as pd
lab = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Lab.csv')
lab.shape

(21641523, 11)

In [21]:
lab

,LabNo,PatientUID,PatientVisitUID,Location,RequestItemName,ResultItemName,CWhen,ResultDttm,ResultValue,Unit,ReferenceRange
0,1,2299,2024049,เวชศาสตร์นิวเคลียร์ (OPD)-D15,BMD LUMBAR,Formatted Result,2558-02-19 16:16:51.423,NaN,FT,NaN,NaN
1,1,2299,2024049,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,"ALBUMIN, 32403","ALBUMIN, 32403",2558-02-19 10:32:20.907,NaN,4.1,NaN,3.8 - 5.4
2,1,2299,2024049,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,Alkaline Phos,"ALP, 32309",2558-02-19 10:32:21.610,NaN,44,NaN,42 - 121
3,1,2299,2024049,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,"ALT, 32311","ALT, 32311",2558-02-19 10:32:21.390,NaN,17,NaN,4 - 36
4,1,2299,2024049,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,"AST, 32310","AST, 32310",2558-02-19 10:32:21.500,NaN,23,NaN,12 - 32
...,...,...,...,...,...,...,...,...,...,...,...
21641153,601,822860,5140446,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,Dextrostix(POCT),DTX(POCT),2561-10-19 11:00:42.207,NaN,228,mg/dL,-
21641156,602,822860,5140446,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,Dextrostix(POCT),DTX(POCT),2561-10-19 17:24:45.417,NaN,240,mg/dL,-
21641159,603,822860,5140446,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,Dextrostix(POCT),DTX(POCT),2561-10-19 22:40:42.073,NaN,171,mg/dL,-
21641162,604,822860,5140446,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,Dextrostix(POCT),DTX(POCT),2561-10-20 05:39:12.677,NaN,191,mg/dL,-




> ***ข้อมูลผู้ป่วยตั้งแต่ 01-12-2557 ถึง 10-04-2566**



In [22]:
lab.sort_values('CWhen')

,LabNo,PatientUID,PatientVisitUID,Location,RequestItemName,ResultItemName,CWhen,ResultDttm,ResultValue,Unit,ReferenceRange
3797953,1,446648,1839345,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,Complete blood count,RDW,2557-12-01 06:55:29.080,NaN,28.5,%,NaN
3797952,1,446648,1839345,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,Complete blood count,RBC,2557-12-01 06:55:29.080,NaN,03.31,NaN,NaN
3797951,1,446648,1839345,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,Complete blood count,Poikilo,2557-12-01 06:55:29.080,NaN,Few,NaN,NaN
3797950,1,446648,1839345,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,Complete blood count,Plt smear,2557-12-01 06:55:29.080,NaN,Adequate,NaN,NaN
3797949,1,446648,1839345,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,Complete blood count,PLT,2557-12-01 06:55:29.080,NaN,0219,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1564803,1,132134,10427090,ห้องปฏิบัติการหน่วยจุลชีววิทยาคลินิก,Aerobic culture and sensitivity : TRACHEAL SUC...,Aerobic culture and sensitivity : TRACHEAL SUC...,2566-04-10 16:28:54.843,NaN,FT,NaN,NaN
11770242,1,1845798,10564519,ห้องปฏิบัติการภาควิชาพยาธิวิทยา,ชิ้นเนื้อเล็กไม่เกิน 2 ซม.,NaN,2566-04-10 16:33:02.827,NaN,FT,NaN,NaN
4468199,1,565443,10572514,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,DIRECT BILIRUBIN,"DIRECT BILIRUBIN, 32207",2566-04-10 16:33:58.130,NaN,0.1,mg/dL,0-0.5
4468198,1,565443,10572514,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,CREATININE IN RANDOM URINE,"CREATININE IN RANDOM URINE, 34101",2566-04-10 16:33:59.003,NaN,27.8,mg/dL,40-278


In [5]:
# Create a list from hemo['PatientUID']
patient_uid_list = hemo['PatientUID'].unique().tolist()

In [6]:
# Remove patient UIDs from lab['PatientUID'] that are in the list
lab = lab1[~lab1['PatientUID'].isin(patient_uid_list)]
lab.shape

(17728574, 11)

In [7]:
lab['PatientVisitUID'].nunique() #มีผู้ป่วย 20967 ที่ไม่ฟอกไต เป็นผู้ป่วยที่เราจะใช้

489786

##**🗂️ Patient Data eGFR age >= 18**

In [8]:
# Define the values to match
result_item_names = ['eGFR(CKD-EPI) age >=18']

# Select rows where 'ResultItemName' matches the specified values
eGFR1 = lab1[lab1['ResultItemName'].isin(result_item_names)]
eGFR1
labeGFR1 = eGFR1[['LabNo','PatientUID', 'PatientVisitUID', 'ResultItemName', 'ResultValue', 'CWhen']]
labeGFR1

,LabNo,PatientUID,PatientVisitUID,ResultItemName,ResultValue,CWhen
248,1,2299,3386255,eGFR(CKD-EPI) age >=18,71.7,2559-08-15 09:51:54.913
255,1,2299,3640924,eGFR(CKD-EPI) age >=18,66.8,2559-11-11 09:40:06.053
271,1,2299,4020970,eGFR(CKD-EPI) age >=18,67.20,2560-03-31 10:18:54.363
286,1,2299,4286612,eGFR(CKD-EPI) age >=18,69.20,2560-07-05 10:40:40.763
331,1,2299,4546471,eGFR(CKD-EPI) age >=18,63.60,2560-10-03 09:19:54.710
...,...,...,...,...,...,...
21628049,159,925649,3617725,eGFR(CKD-EPI) age >=18,77.60,2560-04-17 08:54:35.740
21628297,160,925649,3617725,eGFR(CKD-EPI) age >=18,72.00,2560-04-19 08:41:45.060
21628538,161,925649,3617725,eGFR(CKD-EPI) age >=18,62.90,2560-04-21 08:47:03.167
21628775,162,925649,3617725,eGFR(CKD-EPI) age >=18,59.60,2560-04-24 08:59:53.310


In [9]:
import re
# Define the regular expression pattern for valid values (digits and decimals)
pattern = r'^[0-9]+(\.[0-9]+)?$'

# Filter rows with non-digit or non-decimal values in the 'ResultValue' column
non_digit_rows1 = labeGFR1[~labeGFR1['ResultValue'].astype(str).apply(lambda x: bool(re.match(pattern, x)))]
non_digit_rows1 #64 rows before hemo ?? why we could not see this problem??

,LabNo,PatientUID,PatientVisitUID,ResultItemName,ResultValue,CWhen
1330622,1,119514,3571540,eGFR(CKD-EPI) age >=18,ND,2559-10-18 20:48:28.817
4453496,1,563983,3804017,eGFR(CKD-EPI) age >=18,-,2560-01-12 13:04:14.380
4471358,1,566187,8235036,eGFR(CKD-EPI) age >=18,Not detected,2564-03-19 09:09:07.297
5232553,1,690624,7797789,eGFR(CKD-EPI) age >=18,ND,2563-10-21 16:16:58.290
6486667,1,871360,4278938,eGFR(CKD-EPI) age >=18,nd,2560-07-03 15:04:50.793
...,...,...,...,...,...,...
20999454,28,464537,4882028,eGFR(CKD-EPI) age >=18,cancle,2561-02-28 09:50:42.277
21191192,35,738697,5438058,eGFR(CKD-EPI) age >=18,-,2561-09-07 08:31:36.957
21215351,36,1168494,9701496,eGFR(CKD-EPI) age >=18,xx,2565-09-01 07:56:11.627
21260028,39,22211,3392590,eGFR(CKD-EPI) age >=18,-,2559-10-02 08:52:02.277


In [10]:
# Define the values to match
result_item_names = ['eGFR(CKD-EPI) age >=18']

# Select rows where 'ResultItemName' matches the specified values
eGFR = lab[lab['ResultItemName'].isin(result_item_names)]
eGFR
labeGFR = eGFR[['LabNo','PatientUID', 'PatientVisitUID', 'ResultItemName', 'ResultValue', 'CWhen']]
labeGFR

,LabNo,PatientUID,PatientVisitUID,ResultItemName,ResultValue,CWhen
248,1,2299,3386255,eGFR(CKD-EPI) age >=18,71.7,2559-08-15 09:51:54.913
255,1,2299,3640924,eGFR(CKD-EPI) age >=18,66.8,2559-11-11 09:40:06.053
271,1,2299,4020970,eGFR(CKD-EPI) age >=18,67.20,2560-03-31 10:18:54.363
286,1,2299,4286612,eGFR(CKD-EPI) age >=18,69.20,2560-07-05 10:40:40.763
331,1,2299,4546471,eGFR(CKD-EPI) age >=18,63.60,2560-10-03 09:19:54.710
...,...,...,...,...,...,...
21628049,159,925649,3617725,eGFR(CKD-EPI) age >=18,77.60,2560-04-17 08:54:35.740
21628297,160,925649,3617725,eGFR(CKD-EPI) age >=18,72.00,2560-04-19 08:41:45.060
21628538,161,925649,3617725,eGFR(CKD-EPI) age >=18,62.90,2560-04-21 08:47:03.167
21628775,162,925649,3617725,eGFR(CKD-EPI) age >=18,59.60,2560-04-24 08:59:53.310


In [11]:
import re
# Define the regular expression pattern for valid values (digits and decimals)
pattern = r'^[0-9]+(\.[0-9]+)?$'

# Filter rows with non-digit or non-decimal values in the 'ResultValue' column
non_digit_rows = labeGFR[~labeGFR['ResultValue'].astype(str).apply(lambda x: bool(re.match(pattern, x)))]
non_digit_rows  #51 rows have eGFR as Not Deteched

,LabNo,PatientUID,PatientVisitUID,ResultItemName,ResultValue,CWhen
1330622,1,119514,3571540,eGFR(CKD-EPI) age >=18,ND,2559-10-18 20:48:28.817
4453496,1,563983,3804017,eGFR(CKD-EPI) age >=18,-,2560-01-12 13:04:14.380
4471358,1,566187,8235036,eGFR(CKD-EPI) age >=18,Not detected,2564-03-19 09:09:07.297
6486667,1,871360,4278938,eGFR(CKD-EPI) age >=18,nd,2560-07-03 15:04:50.793
9528830,1,1269185,6082408,eGFR(CKD-EPI) age >=18,cancel,2562-03-13 09:40:29.180
9566531,1,1276548,6350160,eGFR(CKD-EPI) age >=18,ND,2562-06-12 08:32:21.860
9642442,1,1289020,6842443,eGFR(CKD-EPI) age >=18,nd,2562-11-15 12:28:30.490
10054359,1,1370560,6297595,eGFR(CKD-EPI) age >=18,nd,2562-05-27 13:58:44.777
10667958,1,1480125,3615923,eGFR(CKD-EPI) age >=18,-,2559-11-02 10:43:49.273
10935965,1,1523723,7139521,eGFR(CKD-EPI) age >=18,ND,2564-01-06 14:54:19.083


In [12]:
import pandas as pd

# Assuming you have a DataFrame named 'eGFR' with the specified columns
pivot_table = pd.pivot_table(labeGFR, index=['PatientUID', 'PatientVisitUID'], values=['ResultItemName', 'ResultValue', 'CWhen'], aggfunc='first')
pivot_table

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2299       3386255          2559-08-15 09:51:54.913  eGFR(CKD-EPI) age >=18   
           3640924          2559-11-11 09:40:06.053  eGFR(CKD-EPI) age >=18   
           4020970          2560-03-31 10:18:54.363  eGFR(CKD-EPI) age >=18   
           4286612          2560-07-05 10:40:40.763  eGFR(CKD-EPI) age >=18   
           4546471          2560-10-03 09:19:54.710  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3432056    10312896         2566-01-30 16:16:45.013  eGFR(CKD-EPI) age >=18   
           10408954         2566-02-24 21:05:12.747  eGFR(CKD-EPI) age >=18   
           10573233         2566-04-10 09:51:32.587  eGFR(CKD-EPI) age >=18   
3437526    10191122         2565-12-26 09:29:11.307  eGFR(CKD-EPI) age >=18   
           10541118         2566-03-30 18:44:40.160  eGFR(CKD-EPI) age >=18   

                           ResultValue  
PatientUID PatientVisitUID              
2299       3386255                71.7  
           3640924                66.8  
           4020970               67.20  
           4286612               69.20  
           4546471               63.60  
...                                ...  
3432056    10312896              72.86  
           10408954              68.69  
           10573233              45.08  
3437526    10191122              55.94  
           10541118              56.99  

[318686 rows x 3 columns]

In [ ]:
pivot_table.isnull().any()

CWhen             False
ResultItemName    False
ResultValue       False
dtype: bool

##  **🕑 generate datetime**
    Split CWhen เพราะต้องการ generate year as คศ. Pandas มี outbound
    ให้อยู่ในระยะเวลาของ คศ. แต่ลอง convert column 'CWhen'
    เป็น datetime แล้วทำไม่ได้เลยต้อง split แล้วค่อย merge

In [13]:
import pandas as pd

df = pd.DataFrame(pivot_table)

# Splitting the 'time' column into separate columns
df[['date', 'timestamp']] = df['CWhen'].str.split(' ', expand=True)
df[['year', 'month', 'day']] = df['date'].str.split('-', expand=True)
df[['hour', 'minute', 'second']] = df['timestamp'].str.split(':', expand=True)
df[['second', 'nanosecond']] = df['second'].str.split('.', expand=True)

# Convert year column from BE to AD
df['year'] = df['year'].astype(int) - 543

# Combine columns into a datetime column
df['date'] = pd.to_datetime(df[['year', 'month', 'day',]])
# Convert timestamp to Timedelta
df['timestamp'] = pd.to_timedelta(df['timestamp'])

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

# Merge timestamp and date columns
df['datetime'] = df['date'] + df['timestamp']

df_sorted = df.sort_values(by=['PatientUID', 'datetime'])
df_sorted.isnull().any()

CWhen             False
ResultItemName    False
ResultValue       False
date              False
timestamp         False
year              False
month             False
day               False
hour              False
minute            False
second            False
nanosecond        False
datetime          False
dtype: bool

In [19]:
a = df.reset_index().sort_values('date')

,PatientUID,PatientVisitUID,CWhen,ResultItemName,ResultValue,date,timestamp,year,month,day,hour,minute,second,nanosecond,datetime
109965,622458,3340846,2559-07-28 16:10:25.433,eGFR(CKD-EPI) age >=18,63.6,2016-07-28,0 days 16:10:25.433000,2016,07,28,16,10,25,433,2016-07-28 16:10:25.433
164054,966461,3343522,2559-07-29 15:19:09.533,eGFR(CKD-EPI) age >=18,32.6,2016-07-29,0 days 15:19:09.533000,2016,07,29,15,19,09,533,2016-07-29 15:19:09.533
592,4515,3344891,2559-07-29 13:09:05.737,eGFR(CKD-EPI) age >=18,53.3,2016-07-29,0 days 13:09:05.737000,2016,07,29,13,09,05,737,2016-07-29 13:09:05.737
268941,2117695,3309832,2559-07-29 12:14:23.720,eGFR(CKD-EPI) age >=18,63.7,2016-07-29,0 days 12:14:23.720000,2016,07,29,12,14,23,720,2016-07-29 12:14:23.720
275648,2266309,3335776,2559-07-29 12:13:38.107,eGFR(CKD-EPI) age >=18,36.9,2016-07-29,0 days 12:13:38.107000,2016,07,29,12,13,38,107,2016-07-29 12:13:38.107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212507,1263134,10571617,2566-04-10 07:42:00.607,eGFR(CKD-EPI) age >=18,58.03,2023-04-10,0 days 07:42:00.607000,2023,04,10,07,42,00,607,2023-04-10 07:42:00.607
272595,2194098,10572312,2566-04-10 08:23:44.373,eGFR(CKD-EPI) age >=18,29.44,2023-04-10,0 days 08:23:44.373000,2023,04,10,08,23,44,373,2023-04-10 08:23:44.373
272005,2180110,10574950,2566-04-10 14:56:44.790,eGFR(CKD-EPI) age >=18,72.45,2023-04-10,0 days 14:56:44.790000,2023,04,10,14,56,44,790,2023-04-10 14:56:44.790
276171,2278604,10573321,2566-04-10 10:18:29.913,eGFR(CKD-EPI) age >=18,60.40,2023-04-10,0 days 10:18:29.913000,2023,04,10,10,18,29,913,2023-04-10 10:18:29.913




> **Lag Generate**

```
 for id in patientID:
    validVisitID = [visit0]
    visitBefore = visit0
    for visit in patientVisit[1:]:
        visitNow = visit
        if timeVisitNow - timeVisitBefore >= 7:
               validVisitID.append(visit)
               visitBefore = visit

```



In [ ]:
import pandas as pd

# Step 1: Reset the index to perform calculations
df_sorted_reset = df_sorted.reset_index()

# Step 2: Create lag variables (lag 1 to lag 6) for the 'date' column within each patient
lag_periods = 6
for lag in range(1, lag_periods + 1):
    df_sorted_reset[f'date_lag_{lag}'] = df_sorted_reset.groupby('PatientUID')['date'].shift(lag)

# Step 3: Apply the condition to check for validVisitID within each patient
validVisitID_list = []
for patientID, patient_df in df_sorted_reset.groupby('PatientUID'):

    validVisitID = [patient_df.iloc[0]['PatientVisitUID']]  # Initialize with the first visit
    visitBefore = patient_df.iloc[0]['datetime']

    for _, row in patient_df.iloc[1:].iterrows():
        visitNow = row['datetime']
        if visitNow - visitBefore >= pd.Timedelta(days=7):
            validVisitID.append(row['PatientVisitUID'])
            visitBefore = visitNow

    validVisitID_list.extend(validVisitID)

# Step 4: Filter the DataFrame based on the validVisitID
df_valid_visits = df_sorted_reset[df_sorted_reset['PatientVisitUID'].isin(validVisitID_list)]

# Step 5: Set the index back to ['PatientUID', 'PatientVisitUID']
df_valid_visits.set_index(['PatientUID', 'PatientVisitUID'], inplace=True)

In [ ]:
df_valid_visits.isnull().any()

CWhen             False
ResultItemName    False
ResultValue       False
date              False
timestamp         False
year              False
month             False
day               False
hour              False
minute            False
second            False
nanosecond        False
datetime          False
date_lag_1         True
date_lag_2         True
date_lag_3         True
date_lag_4         True
date_lag_5         True
date_lag_6         True
dtype: bool

In [ ]:
len(set(validVisitID_list))

305172



> **Calculate TimeDiff and Slope**

In [ ]:
# Step 6: Generate a new column 'TimeDiff' representing the time difference from 'datetime'
df_valid_visits['TimeDiff'] = df_valid_visits['datetime'].diff().dt.days

#Check If there rows that TimeDiff <7
df_valid_visits[df_valid_visits['TimeDiff'] < 7] ### its the first row of each patientvisitid so it calculate time_diff from another datetime's patient we have to drop all of this

<ipython-input-18-2a4cc01af89b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_visits['TimeDiff'] = df_valid_visits['datetime'].diff().dt.days


,,CWhen,ResultItemName,ResultValue,date,timestamp,year,month,day,hour,minute,second,nanosecond,datetime,date_lag_1,date_lag_2,date_lag_3,date_lag_4,date_lag_5,date_lag_6,TimeDiff
PatientUID,PatientVisitUID,,,,,,,,,,,,,,,,,,,,
2304,3431160,2559-08-30 09:37:00.867,eGFR(CKD-EPI) age >=18,68.9,2016-08-30,0 days 09:37:00.867000,2016,08,30,09,37,00,867,2016-08-30 09:37:00.867,NaT,NaT,NaT,NaT,NaT,NaT,-2257.0
2312,5136829,2561-04-28 14:39:32.587,eGFR(CKD-EPI) age >=18,57.60,2018-04-28,0 days 14:39:32.587000,2018,04,28,14,39,32,587,2018-04-28 14:39:32.587,NaT,NaT,NaT,NaT,NaT,NaT,-1719.0
2354,7003273,2563-01-09 07:57:21.190,eGFR(CKD-EPI) age >=18,56.99,2020-01-09,0 days 07:57:21.190000,2020,01,09,07,57,21,190,2020-01-09 07:57:21.190,NaT,NaT,NaT,NaT,NaT,NaT,-1145.0
2362,3910608,2560-02-20 19:48:50.243,eGFR(CKD-EPI) age >=18,67.60,2017-02-20,0 days 19:48:50.243000,2017,02,20,19,48,50,243,2017-02-20 19:48:50.243,NaT,NaT,NaT,NaT,NaT,NaT,-2229.0
2378,3420850,2559-08-26 09:00:24.393,eGFR(CKD-EPI) age >=18,25.9,2016-08-26,0 days 09:00:24.393000,2016,08,26,09,00,24,393,2016-08-26 09:00:24.393,NaT,NaT,NaT,NaT,NaT,NaT,-2399.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3421578,10151787,2565-12-14 11:30:53.310,eGFR(CKD-EPI) age >=18,47.44,2022-12-14,0 days 11:30:53.310000,2022,12,14,11,30,53,310,2022-12-14 11:30:53.310,NaT,NaT,NaT,NaT,NaT,NaT,-92.0
3426284,10105952,2565-11-29 10:47:37.917,eGFR(CKD-EPI) age >=18,27.88,2022-11-29,0 days 10:47:37.917000,2022,11,29,10,47,37,917,2022-11-29 10:47:37.917,NaT,NaT,NaT,NaT,NaT,NaT,-106.0
3431501,10144649,2565-12-13 13:42:48.510,eGFR(CKD-EPI) age >=18,54.65,2022-12-13,0 days 13:42:48.510000,2022,12,13,13,42,48,510,2022-12-13 13:42:48.510,NaT,NaT,NaT,NaT,NaT,NaT,-87.0


In [ ]:
df_valid_visits.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 305172 entries, (2299, 3386255) to (3437526, 10541118)
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   CWhen           305172 non-null  object         
 1   ResultItemName  305172 non-null  object         
 2   ResultValue     305172 non-null  object         
 3   date            305172 non-null  datetime64[ns] 
 4   timestamp       305172 non-null  timedelta64[ns]
 5   year            305172 non-null  int64          
 6   month           305172 non-null  object         
 7   day             305172 non-null  object         
 8   hour            305172 non-null  object         
 9   minute          305172 non-null  object         
 10  second          305172 non-null  object         
 11  nanosecond      305172 non-null  object         
 12  datetime        305172 non-null  datetime64[ns] 
 13  date_lag_1      284205 non-null  datetime64[ns]

In [ ]:
a = df_valid_visits.reset_index()

In [ ]:
a[(a['PatientUID'] == 563983) & (a['PatientVisitUID'] == 3804017)]

,PatientUID,PatientVisitUID,CWhen,ResultItemName,ResultValue,date,timestamp,year,month,day,...,second,nanosecond,datetime,date_lag_1,date_lag_2,date_lag_3,date_lag_4,date_lag_5,date_lag_6,TimeDiff
97005,563983,3804017,2560-01-12 13:04:14.380,eGFR(CKD-EPI) age >=18,-,2017-01-12,0 days 13:04:14.380000,2017,01,12,...,14,380,2017-01-12 13:04:14.380,2016-09-22,NaT,NaT,NaT,NaT,NaT,112.0


In [ ]:
a[(a['PatientUID'] == 119514) & (a['PatientVisitUID'] == 3571540)]

,PatientUID,PatientVisitUID,CWhen,ResultItemName,ResultValue,date,timestamp,year,month,day,...,second,nanosecond,datetime,date_lag_1,date_lag_2,date_lag_3,date_lag_4,date_lag_5,date_lag_6,TimeDiff
30817,119514,3571540,2559-10-18 20:48:28.817,eGFR(CKD-EPI) age >=18,ND,2016-10-18,0 days 20:48:28.817000,2016,10,18,...,28,817,2016-10-18 20:48:28.817,2016-09-27,NaT,NaT,NaT,NaT,NaT,21.0


In [ ]:
a[(a['PatientUID'] == 566187) & (a['PatientVisitUID'] == 8235036)]

,PatientUID,PatientVisitUID,CWhen,ResultItemName,ResultValue,date,timestamp,year,month,day,...,second,nanosecond,datetime,date_lag_1,date_lag_2,date_lag_3,date_lag_4,date_lag_5,date_lag_6,TimeDiff
97449,566187,8235036,2564-03-19 09:09:07.297,eGFR(CKD-EPI) age >=18,Not detected,2021-03-19,0 days 09:09:07.297000,2021,03,19,...,07,297,2021-03-19 09:09:07.297,2020-08-12,2020-06-20,2020-01-28,2019-06-14,2019-01-31,2018-01-31,218.0


In [ ]:
a[(a['PatientUID'] == 1289020) & (a['PatientVisitUID'] == 6842443)]

,PatientUID,PatientVisitUID,CWhen,ResultItemName,ResultValue,date,timestamp,year,month,day,...,second,nanosecond,datetime,date_lag_1,date_lag_2,date_lag_3,date_lag_4,date_lag_5,date_lag_6,TimeDiff
206552,1289020,6842443,2562-11-15 12:28:30.490,eGFR(CKD-EPI) age >=18,nd,2019-11-15,0 days 12:28:30.490000,2019,11,15,...,30,490,2019-11-15 12:28:30.490,2019-08-22,2019-06-20,2019-05-02,2019-04-29,2019-04-05,2019-03-28,85.0




> **Drop 11 Rows eGFR => Not Deteched**



In [ ]:
# Convert 'ResultValue' column to numeric data type
a = pd.to_numeric(df_valid_visits['ResultValue'], errors='coerce')
len(a[a.isnull()]) # 11 rows
a[a.isnull()]

PatientUID  PatientVisitUID
119514      3571540           NaN
563983      3804017           NaN
566187      8235036           NaN
871360      4278938           NaN
1269185     6082408           NaN
1276548     6350160           NaN
1289020     6842443           NaN
1370560     6297595           NaN
1480125     3615923           NaN
1523723     7139521           NaN
1558914     3737686           NaN
2266212     6516947           NaN
2587904     5828463           NaN
Name: ResultValue, dtype: float64

In [ ]:

# Convert 'ResultValue' and 'TimeDiff' columns to numeric data types
df_valid_visits['ResultValue'] = pd.to_numeric(df_valid_visits['ResultValue'], errors='coerce')
df_valid_visits['TimeDiff'] = pd.to_numeric(df_valid_visits['TimeDiff'], errors='coerce')

# Drop rows where 'TimeDiff' is NaN, zero, or negative (to avoid division by zero or negative values)
df_valid_visits = df_valid_visits[df_valid_visits['ResultValue'].notna() & (df_valid_visits['ResultValue'] > 0)]
df_valid_visits = df_valid_visits[df_valid_visits['TimeDiff'].notna() & (df_valid_visits['TimeDiff'] > 0)]

# Calculate the slope between 'ResultValue' and 'TimeDiff'
# Note: This is a simple calculation, and you may want to modify it based on your specific requirements
df_valid_visits['Slope'] = df_valid_visits['ResultValue'] / df_valid_visits['TimeDiff']



<ipython-input-26-fbbd3c33f466>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_visits['ResultValue'] = pd.to_numeric(df_valid_visits['ResultValue'], errors='coerce')
<ipython-input-26-fbbd3c33f466>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_visits['TimeDiff'] = pd.to_numeric(df_valid_visits['TimeDiff'], errors='coerce')


In [ ]:
df_valid_visits[df_valid_visits['ResultValue'].isnull() & (df_valid_visits['ResultValue'] < 0)]

,,CWhen,ResultItemName,ResultValue,date,timestamp,year,month,day,hour,minute,...,nanosecond,datetime,date_lag_1,date_lag_2,date_lag_3,date_lag_4,date_lag_5,date_lag_6,TimeDiff,Slope
PatientUID,PatientVisitUID,,,,,,,,,,,,,,,,,,,,,


In [ ]:
df_valid_visits.isnull().any()

CWhen             False
ResultItemName    False
ResultValue       False
date              False
timestamp         False
year              False
month             False
day               False
hour              False
minute            False
second            False
nanosecond        False
datetime          False
date_lag_1         True
date_lag_2         True
date_lag_3         True
date_lag_4         True
date_lag_5         True
date_lag_6         True
TimeDiff          False
Slope             False
dtype: bool



> **CHECK**

```
*   inf row
*   negative time_diff
```




In [ ]:
import numpy as np
import pandas as pd

# Assuming you have already calculated 'Slope' as shown in previous steps

# Find rows with infinite 'Slope' values
inf_rows = df_valid_visits[np.isinf(df_valid_visits['Slope'])]
inf_indices = inf_rows.index

# Find rows immediately before the rows with infinite 'Slope' values
rows_before_inf = df_valid_visits[np.isinf(df_valid_visits['Slope'].shift(-1))]

# Concatenate both DataFrames to get rows before and after 'inf_rows'
result = pd.concat([rows_before_inf, inf_rows])

# Print the resulting DataFrame
result

,,CWhen,ResultItemName,ResultValue,date,timestamp,year,month,day,hour,minute,...,nanosecond,datetime,date_lag_1,date_lag_2,date_lag_3,date_lag_4,date_lag_5,date_lag_6,TimeDiff,Slope
PatientUID,PatientVisitUID,,,,,,,,,,,,,,,,,,,,,


In [ ]:
df_valid_visits[df_valid_visits['Slope']<=0]


,,CWhen,ResultItemName,ResultValue,date,timestamp,year,month,day,hour,minute,...,nanosecond,datetime,date_lag_1,date_lag_2,date_lag_3,date_lag_4,date_lag_5,date_lag_6,TimeDiff,Slope
PatientUID,PatientVisitUID,,,,,,,,,,,,,,,,,,,,,


In [ ]:
df = pd.DataFrame(df_valid_visits)
df

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2299       3640924          2559-11-11 09:40:06.053  eGFR(CKD-EPI) age >=18   
           4020970          2560-03-31 10:18:54.363  eGFR(CKD-EPI) age >=18   
           4286612          2560-07-05 10:40:40.763  eGFR(CKD-EPI) age >=18   
           4546471          2560-10-03 09:19:54.710  eGFR(CKD-EPI) age >=18   
           4801938          2561-01-04 09:03:28.847  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3432056    10235244         2566-01-09 12:31:53.220  eGFR(CKD-EPI) age >=18   
           10306813         2566-01-27 12:22:43.000  eGFR(CKD-EPI) age >=18   
           10408954         2566-02-24 21:05:12.747  eGFR(CKD-EPI) age >=18   
           10573233         2566-04-10 09:51:32.587  eGFR(CKD-EPI) age >=18   
3437526    10541118         2566-03-30 18:44:40.160  eGFR(CKD-EPI) age >=18   

                            ResultValue       date              timestamp  \
PatientUID PatientVisitUID                                                  
2299       3640924                66.80 2016-11-11 0 days 09:40:06.053000   
           4020970                67.20 2017-03-31 0 days 10:18:54.363000   
           4286612                69.20 2017-07-05 0 days 10:40:40.763000   
           4546471                63.60 2017-10-03 0 days 09:19:54.710000   
           4801938                58.40 2018-01-04 0 days 09:03:28.847000   
...                                 ...        ...                    ...   
3432056    10235244               48.51 2023-01-09 0 days 12:31:53.220000   
           10306813               67.71 2023-01-27        0 days 12:22:43   
           10408954               68.69 2023-02-24 0 days 21:05:12.747000   
           10573233               45.08 2023-04-10 0 days 09:51:32.587000   
3437526    10541118               56.99 2023-03-30 0 days 18:44:40.160000   

                            year month day hour minute  ... nanosecond  \
PatientUID PatientVisitUID                              ...              
2299       3640924          2016    11  11   09     40  ...        053   
           4020970          2017    03  31   10     18  ...        363   
           4286612          2017    07  05   10     40  ...        763   
           4546471          2017    10  03   09     19  ...        710   
           4801938          2018    01  04   09     03  ...        847   
...                          ...   ...  ..  ...    ...  ...        ...   
3432056    10235244         2023    01  09   12     31  ...        220   
           10306813         2023    01  27   12     22  ...        000   
           10408954         2023    02  24   21     05  ...        747   
           10573233         2023    04  10   09     51  ...        587   
3437526    10541118         2023    03  30   18     44  ...        160   

                                          datetime date_lag_1 date_lag_2  \
PatientUID PatientVisitUID                                                 
2299       3640924         2016-11-11 09:40:06.053 2016-08-15        NaT   
           4020970         2017-03-31 10:18:54.363 2016-11-11 2016-08-15   
           4286612         2017-07-05 10:40:40.763 2017-03-31 2016-11-11   
           4546471         2017-10-03 09:19:54.710 2017-07-05 2017-03-31   
           4801938         2018-01-04 09:03:28.847 2017-10-03 2017-07-05   
...                                            ...        ...        ...   
3432056    10235244        2023-01-09 12:31:53.220 2022-12-15        NaT   
           10306813        2023-01-27 12:22:43.000 2023-01-09 2022-12-15   
           10408954        2023-02-24 21:05:12.747 2023-01-30 2023-01-27   
           10573233        2023-04-10 09:51:32.587 2023-02-24 2023-01-30   
3437526    10541118        2023-03-30 18:44:40.160 2022-12-26        NaT   

                           date_lag_3 date_lag_4 date_lag_5 date

In [ ]:
df = df[['ResultValue','datetime','TimeDiff', 'Slope']]
df

ResultValue                datetime  TimeDiff  \
PatientUID PatientVisitUID                                                  
2299       3640924                66.80 2016-11-11 09:40:06.053      87.0   
           4020970                67.20 2017-03-31 10:18:54.363     140.0   
           4286612                69.20 2017-07-05 10:40:40.763      96.0   
           4546471                63.60 2017-10-03 09:19:54.710      89.0   
           4801938                58.40 2018-01-04 09:03:28.847      92.0   
...                                 ...                     ...       ...   
3432056    10235244               48.51 2023-01-09 12:31:53.220      25.0   
           10306813               67.71 2023-01-27 12:22:43.000      17.0   
           10408954               68.69 2023-02-24 21:05:12.747      28.0   
           10573233               45.08 2023-04-10 09:51:32.587      44.0   
3437526    10541118               56.99 2023-03-30 18:44:40.160      94.0   

                               Slope  
PatientUID PatientVisitUID            
2299       3640924          0.767816  
           4020970          0.480000  
           4286612          0.720833  
           4546471          0.714607  
           4801938          0.634783  
...                              ...  
3432056    10235244         1.940400  
           10306813         3.982941  
           10408954         2.453214  
           10573233         1.024545  
3437526    10541118         0.606277  

[285170 rows x 4 columns]

In [ ]:
df.isnull().any()

ResultValue    False
datetime       False
TimeDiff       False
Slope          False
dtype: bool


> **Cut data**
```
เช็คว่าแต่ละคนมีจำนวนสโลปที่คำนวณจากระยะเวลาห่างกันอย่างน้อย 3 เดือน
อย่างน้อย 2 สโลปไหม ถ้ามีไม่ครบให้ตัดคนนั้นทิ้ง
```


In [ ]:
import pandas as pd

# Assuming you have a DataFrame 'eGFR_Patient' with index 'PatientUID' and columns 'datetime', 'slopes', and 'time_difference'

# Create an empty list to store the valid PatientUIDs
valid_patient_uids = []

# Group the DataFrame by index 'PatientUID'
grouped = df.groupby(level='PatientUID')

# Iterate over each group
for patient_uid, group in grouped:
    # Check if the group has at least 2 rows
    if len(group) >= 3:
        # Calculate the minimum and maximum datetime in the group
        min_datetime = group['datetime'].min()
        max_datetime = group['datetime'].max()

        # Calculate the time difference in days between the minimum and maximum datetime
        time_diff = (max_datetime - min_datetime).days

        # Check if the time difference is at least 90 days (3 months)
        if time_diff >= 90:
            valid_patient_uids.append(patient_uid)

# Filter the original DataFrame based on the valid PatientUIDs
filtered_df = df.loc[valid_patient_uids]

In [ ]:
filtered_df[filtered_df['Slope']<=0]

,,ResultValue,datetime,TimeDiff,Slope
PatientUID,PatientVisitUID,,,,


In [ ]:
filtered_df[filtered_df['Slope']==float('inf')]

,,ResultValue,datetime,TimeDiff,Slope
PatientUID,PatientVisitUID,,,,


In [ ]:
filtered_df.reset_index()['PatientUID'].nunique()

17711

In [ ]:
a = filtered_df.reset_index()

In [ ]:
a.isnull().any()

PatientUID         False
PatientVisitUID    False
ResultValue        False
datetime           False
TimeDiff           False
Slope              False
dtype: bool

In [ ]:
a.to_excel('LabSlope.xlsx', index=False)

In [ ]:
import re
# Define the regular expression pattern for valid values (digits and decimals)
pattern = r'^[0-9]+(\.[0-9]+)?$'

# Filter rows with non-digit or non-decimal values in the 'ResultValue' column
non_digit_rows = a[~a['ResultValue'].astype(str).apply(lambda x: bool(re.match(pattern, x)))]
non_digit_rows

,PatientUID,PatientVisitUID,ResultValue,datetime,TimeDiff,Slope
